In [41]:
import math
import json

In [4]:
fread = open("/content/Reddit_data_train.json",'r')
data = json.load(fread)


In [5]:
users = []
for i in data.keys():
  if i not in users:
    users.append(i)
  #print(i,len(i),sep=",")
print(len(users))

371


In [6]:
data[users[2]]


[{'text': "&gt; I think the biggest thing that is killing the 'extra' $1000 per month is that my job requires me to drive to many places so I 1. Eat on the go a lot and 2. End up paying $200+ a week in gas and refueling every other day at times. \n\n1. buy a personal sized cooler and pack a lunch &amp; snacks to keep in your car with an ice pack.\n2. your job isn't reimbursing you for your fuel when you have to drive everywhere for your work?  why not?"},
 {'text': "&gt; I also don't think I've realized how babied I am :). When I lived with family, I was always taken care by my family. When I lived with roommates (that were my friends) or my boyfriend (now my ex), they would take care of me when I'm sick or like little things like that. Right now, I don't have much friends here, so having that good relationship is up in the air. \n\nif you think random roommates are going to baby you when you're feeling sick or just feeling lazy, then you'll be in for a shocking surprise."},
 {'text': 

## Text Preprocessing

In [7]:
import re

## Functions for Text Preprocessing

In [8]:
def remove_spl_chars(s):
  pat = r'[^a-zA-z0-9.,!?/:;\"\'\s]'
  return re.sub(pat, '', s)

def remove_extra_whitespace_tabs(text):
    pattern = r'^\s*|\s\s*'
    return re.sub(pattern, ' ', text).strip()



## Not Removing stopwords
#### Removing common words like "the," "a," and "is" frees up the LLM to focus on more critical words that carry the core meaning of the post.
### PROS
#### This can lead to: Improved accuracy: By eliminating non-informative words, the LLM may identify the main topic of the post more accurately. Faster processing: With less data to analyze, the LLM can process the text faster, potentially improving efficiency.

### CONS:
#### Loss of context and nuance: Stop words can sometimes contribute to the overall meaning and sentiment of a sentence. Removing them entirely can lead to: Misinterpretation: The LLM might miss subtle cues or context clues that rely on stop words. Loss of emotional tone: Stop words can play a role in conveying emotions and sentiment, which might be lost without them.

### Thus using a powerful LLM like *PaLM* or *Bard*, they might be able to understand the context even with stop words present.

In [32]:
for us in users:
  for i in range(len(data[us])):
    # print(data[us][i]["text"])
    data[us][i]["text"].replace("\n","")
  # print(txt["text"])
    data[us][i]["text"] = data[us][i]["text"].replace("&gt;","")
  #print(txt["text"])
    data[us][i]["text"] = remove_extra_whitespace_tabs(data[us][i]["text"])
    data[us][i]["text"] = data[us][i]["text"].lower()

In [36]:
data[users[300]][1]["text"]

'by "rent out" do you mean that you want to purchase a house and rent it out? or do you mean that you want to rent a house for yourself to live in? your "buy and sell" comment just made it a bit ambiguous. i think i can help you out tho, depending on what you mean.'

In [ ]:
!pip install langchain

In [ ]:
from langchain.embeddings import SentenceTransformerEmbeddings

## Acquiring wikipedia titles

In [ ]:
!pip install mediawikiapi

In [21]:
import requests
candidate_labels = []
S = requests.Session()
catg = ["Earth","Art","Culture","Geography","Finance","Exchange","Travel"]

URL = "https://en.wikipedia.org/w/api.php"

for cat in catg:
  PARAMS = {
    "action": "query",
    "format": "json",
    "list": "allpages",
    "apfrom": cat,
    "aplimit": "10",
  }

  R = S.get(url=URL, params=PARAMS)
  DATA = R.json()

  PAGES = DATA["query"]["allpages"]

  for page in PAGES:
    # print(page["title"])
    candidate_labels.append(page["title"])

In [22]:
len(candidate_labels)

70

## User Profiling

In [19]:
from transformers import pipeline
classifier = pipeline("zero-shot-classification",
                      model="facebook/bart-large-mnli")

In [40]:
candidate = ['travel', 'cooking', 'finance', 'exploration']
res = classifier(data["lilfunky1"][0]["text"],candidate,multiLabel=True)
res

{'sequence': "i think the biggest thing that is killing the 'extra' $1000 per month is that my job requires me to drive to many places so i 1. eat on the go a lot and 2. end up paying $200+ a week in gas and refueling every other day at times. 1. buy a personal sized cooler and pack a lunch &amp; snacks to keep in your car with an ice pack. 2. your job isn't reimbursing you for your fuel when you have to drive everywhere for your work? why not?",
 'labels': ['travel', 'finance', 'exploration', 'cooking'],
 'scores': [0.7097086906433105,
  0.24100540578365326,
  0.042740922421216965,
  0.006545008160173893]}

In [39]:
res["labels"][0]

'travel'

In [61]:
# candidate_labels = ['travel', 'cooking', 'finance', 'exploration']

user_profile = {}
for usr in users[:2]:
  topics = []
  for i in range(len(data[usr])):
    output = classifier(data[usr][i]["text"], candidate_labels, multi_label=True)
    topics_per_post = output['labels'][:10]
    topics.append(topics_per_post)
    # data[usr][i]['topics'] = topics_per_post
  user_profile[usr] = topics


print(user_profile)

{'S1NCL41R': ['Finance', 'Culture&', 'Culture', 'Finance & Commerce', 'Finance.si', 'Exchange-rate', 'Exchange', 'Travel.state.gov', 'Art', "Art'hanesvari"], '[deleted]': ['Finance', 'Culture&', 'Finance & Commerce', 'Finance.si', 'Culture', 'Culture+Travel', 'Finance, Public', 'Travel', "Art'hanesvari", 'Travel-routing system']}


## Model training

In [63]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Choose your preferred pre-trained model for topic classification
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Step 1: Identify key entities and their relationships
step1_prompt = "This text mentions the following entities: {}. They are related to each other in the following way: {}."

# Step 2: Analyze the main theme or purpose
step2_prompt = "The main theme or purpose of this text is likely about {}."

# Step 3: Classify the topic baseado on the analysis
step3_prompt = "Therefore, this text can be classified as a topic related to {}."



tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [78]:
def get_cot_prompt(text):
  # Extract key entities and relationships (can be replaced with named entity recognition or other NLP techniques)
  entities = candidate_labels
  relationships = "i think the biggest thing that is killing the 'extra' $1000 per month is that my job requires me to drive to many places so i 1. eat on the go a lot and 2. end up paying $200+ a week in gas and refueling every other day at times. 1. buy a personal sized cooler and pack a lunch &amp; snacks to keep in your car with an ice pack. 2. your job isn't reimbursing you for your fuel when you have to drive everywhere for your work? why not?"

  # Generate CoT prompt by filling in the templates
  prompt = step1_prompt.format(entities, relationships) + " " + step2_prompt.format("the impact of these policies") + " " + step3_prompt.format("politics")
  return prompt

def predict_topic(text):
  cot_prompt = get_cot_prompt(text)
  encoded_text = tokenizer(cot_prompt + text, return_tensors="pt", max_length=512, truncation=True)
  outputs = model(**encoded_text)
  predicted_label = int(outputs.logits.argmax(-1))

In [79]:
text = 'by "rent out" do you mean that you want to purchase a house and rent it out?'
predicted_topic = predict_topic(text)
print(f"Predicted topic for the text: {predicted_topic}")

Predicted topic for the text: None


## **Post topic Profiling**

In [67]:
fr = open("/content/Reddit_data_test.json",'r')
data_test = json.load(fr)


In [68]:
data_test[0]

{'text': "360 Savings - How to close sub/second account? There was a bunch of articles that before Capital One redesigned the site, that you could open additional sub/second savings accounts and then close them online when you are done (such as if you open one for Christmas gifts, then close it after Christmas). \n\nFor the life of me I can't see how to close a sub/second savings account? Did they remove the feature or am I just missing it? ",
 'GT': [{'dist_from_root': 2, 'user': 'ohn900'},
  {'dist_from_root': 4, 'user': 'ohn900'},
  {'dist_from_root': 6, 'user': 'ohn900'}]}

In [72]:
for i in range(len(data_test)):
  data_test[i]["text"].replace("\n","")
  # print(txt["text"])
  data_test[i]["text"] = data_test[i]["text"].replace("&gt;","")
  #print(txt["text"])
  data_test[i]["text"] = remove_extra_whitespace_tabs(data_test[i]["text"])
  data_test[i]["text"] = data_test[i]["text"].lower()

In [73]:
data_test[0]["text"]

"360 savings - how to close sub/second account? there was a bunch of articles that before capital one redesigned the site, that you could open additional sub/second savings accounts and then close them online when you are done (such as if you open one for christmas gifts, then close it after christmas). for the life of me i can't see how to close a sub/second savings account? did they remove the feature or am i just missing it?"

In [76]:
#assigns topics to each post (5 posts)
post_profile = []
for i in range(5):
  output_post = classifier(data_test[i]["text"], candidate_labels, multi_label=True)
  topics_per_post = output_post['labels'][:10]
  post_profile.append(topics_per_post)

KeyboardInterrupt: 